In [13]:
# all required libraries below
%pip install numpy --quiet
%pip install tonic --quiet
%pip install matplotlib --quiet
%pip install snntorch --quiet
%pip install torch --quiet
%pip install Lempel-Ziv-Complexity --quiet


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [14]:
# All imports go here
import numpy as np
import numpy.lib.recfunctions as rf
import tonic
import matplotlib.pyplot as plt
from IPython.display import HTML
import torch
import torch.nn as nn
import snntorch as snn
from snntorch import surrogate
from snntorch import functional as SF
from snntorch import utils
from lempel_ziv_complexity import lempel_ziv_complexity

In [15]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(device)

cuda


In [16]:
def calculateLZC(events):
    spike_seq = (events['p'] > 0).astype(int).flatten()  # 1 if spike, 0 if inactive
    spike_seq_string = ''.join(map(str, spike_seq.tolist()))
    lz_score = lempel_ziv_complexity(spike_seq_string)
    print(f"LZ complexity: ", lz_score)

In [17]:
# Example on how to calculate lempel ziv complexity for a single piece of data
# 
#  Load in the data. When we do train[1], we are getting one "video" of data. 
dataset_path = '/home/gauravgupta/CMPM118/data'
train = tonic.datasets.DVSGesture(save_to=dataset_path, train=True)
test = tonic.datasets.DVSGesture(save_to=dataset_path, train=False)
events, label = train[8]

calculateLZC(events)




LZ complexity:  7435


In [18]:
w,h=32,32
n_frames=10

In [19]:
# This is used to figure out how many fully connected neurons need to be present in the last layer. This number depends on the w and h values. 

test_input = torch.zeros((1, 2, w, h))  # 2 polarity channels
x = nn.Conv2d(2, 12, 5)(test_input)
x = nn.MaxPool2d(2)(x)
x = nn.Conv2d(12, 32, 5)(x)
x = nn.MaxPool2d(2)(x)
print("Output shape before flatten:", x.shape)
print("Flattened size:", x.numel())
flattenedSize = x.numel()


Output shape before flatten: torch.Size([1, 32, 5, 5])
Flattened size: 800


In [20]:
grad = snn.surrogate.fast_sigmoid(slope=25)
beta = 0.5

net = nn.Sequential(
    nn.Conv2d(2, 12, 5),
    nn.MaxPool2d(2),
    snn.Leaky(beta=beta, spike_grad=grad, init_hidden=True),
    nn.Conv2d(12, 32, 5),
    nn.MaxPool2d(2),
    snn.Leaky(beta=beta, spike_grad=grad, init_hidden=True),
    nn.Flatten(),
    nn.Linear(800, 11),   # make sure 800 matches flattenedSize
    snn.Leaky(beta=beta, spike_grad=grad, init_hidden=True, output=True)
).to(device)

model_path = "results/large/models/Large_Take2.pth"
net.load_state_dict(torch.load(model_path, map_location=device))
net.eval()
print("Model loaded successfully.")


Model loaded successfully.


In [24]:
dataset_path = '/home/gauravgupta/CMPM118/data'

cache_root = f"{dataset_path}/dvsgesture/{w}x{h}_T{n_frames}"

cached_test = tonic.DiskCachedDataset(
    tonic.datasets.DVSGesture(save_to=dataset_path, train=False),
    cache_path=f"{cache_root}/test"
)


# frames, label = cached_test[1]
# ani = tonic.utils.plot_animation(frames)
# print(frames.shape, label)
# HTML(ani.to_jshtml())







# to visualize the LZC and video of one video, uncomment the code below. This is NOT NECESSARY for training OR running.  



# frames, label = cached_train[1]
# ani = tonic.utils.plot_animation(frames)
# print(frames.shape, label)
# HTML(ani.to_jshtml())

# lzc = ""
# lzc_list = []

# binary_frames = (frames > 0).astype(int)
# flat_frames = binary_frames.reshape(binary_frames.shape[0], -1)
# for frame in flat_frames:
#     lzc = "".join(map(str, frame))
#     lzc_list.append(lempel_ziv_complexity(lzc))

# fig, axes = plt.subplots(1, 1, figsize=(18,4))
# axes.plot(lzc_list)
# axes.set_title("LZC Over Time")
# axes.set_xlabel("Frame")
# axes.set_ylabel("LZC")

# plt.show()

In [ ]:
def forward_pass(net, data):
    spk_rec = []
    snn.utils.reset(net)
    with torch.no_grad():
        for t in range(data.size(0)):          # data: [T, 2, H, W]
            x = data[t].unsqueeze(0).to(device) # -> [1, 2, H, W]
            spk_out, _ = net(x)
            spk_rec.append(spk_out)             # [1, 11]
    return torch.stack(spk_rec)  


def predict_sample(frames):
    frames = torch.tensor(frames, dtype=torch.float)  # [T, 2, H, W]
    spk_rec = forward_pass(net, frames)
    counts = spk_rec.sum(0)            # [1, 11]
    return counts.argmax(1).item()


In [23]:
frames, label = cached_test[1]
print(frames.shape)

# ani = tonic.utils.plot_animation(events)
# print(events.shape, label)
# HTML(ani.to_jshtml())


pred = predict_sample(frames)
print(f"True label: {label}, Predicted label: {pred}")


(10, 2, 32, 32)


RuntimeError: mat1 and mat2 shapes cannot be multiplied (32x25 and 800x11)